In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
    .appName("Process Bronze to Silver")\
    .config("spark.hadoop.fs.defaultFS", "hdfs://localhost:9000")\
    .config("spark.sql.legacy.timeParserPolicy", "LEGACY") \
    .getOrCreate()

25/01/02 21:01:35 WARN Utils: Your hostname, hadoop resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
25/01/02 21:01:35 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/01/02 21:01:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
yfinance_bronze_path = "/user/adam_majczyk2001/nifi/bronze_parquet/yfinance/"
news_bronze_path = "/user/adam_majczyk2001/nifi/bronze_parquet/news/"

yfinance_silver_path = "/user/adam_majczyk2001/nifi/silver_parquet/yfinance/"
news_silver_path = "/user/adam_majczyk2001/nifi/silver_parquet/news/"

In [3]:
yfinance_df = spark.read.parquet(yfinance_bronze_path)
news_df = spark.read.parquet(news_bronze_path)

In [4]:
yfinance_df.head()

Row(timestamp='2025-01-02T19:15:11.756914', updates_XOM=[Row(price=107.14, volume=7218404, volatility=0.329, bid_ask_spread=0.0442, market_sentiment=0.702, trading_activity=12.08, timestamp='2025-01-02T19:10:13.632219', source='real'), Row(price=106.72, volume=6986843, volatility=0.331, bid_ask_spread=0.0433, market_sentiment=0.769, trading_activity=11.58, timestamp='2025-01-02T19:10:13.730177', source='simulated'), Row(price=107.14, volume=6986843, volatility=0.331, bid_ask_spread=0.0433, market_sentiment=0.769, trading_activity=11.58, timestamp='2025-01-02T19:10:14.582108', source='real'), Row(price=106.88, volume=7176769, volatility=0.325, bid_ask_spread=0.0441, market_sentiment=0.679, trading_activity=8.75, timestamp='2025-01-02T19:10:15.570387', source='simulated'), Row(price=107.14, volume=7176769, volatility=0.325, bid_ask_spread=0.0441, market_sentiment=0.679, trading_activity=8.75, timestamp='2025-01-02T19:10:16.026325', source='real'), Row(price=107.14, volume=7176769, volati

In [5]:
news_df.head()

Row(title='Ostrzegają przed unijną polityką. "Zagrozi górniczym płacom i przyspieszy likwidację kopalń"', date='2024-11-28 08:41:00', keywords=['GÓRNICY', 'KOPALNIE', 'RYNEK PRACY', 'UNIA EUROPEJSKA', 'WYDOBYCIE WĘGLA', 'RYNEK WĘGLA', 'PRODUKCJA ENERGII', 'POLITYKA KLIMATYCZNA', 'HUTEK BOGUSŁAW', 'GÓRNICTWO', 'ENERGETYKA WĘGLOWA', 'EUROPEJSKI ZIELONY ŁAD'], is_premium=False, source_site='wnp.pl', url='https://www.wnp.pl/energetyka/ostrzegaja-przed-unijna-polityka-zagrozi-gorniczym-placom-i-przyspieszy-likwidacje-kopaln,892344.html', text='Do kopalń trafiła Błyskawica - publikacja Krajowej Sekcji Górnictwa Węgla Kamiennego NSZZ Solidarność poświęcona zagrożeniom, jakie wynikają z Europejskiego Zielonego Ładu. - Wprowadzenie Zielonego Ładu zagrozi istnieniu tysięcy miejsc pracy w sektorze wydobywczym - ocenia Bogusław Hutek, szef górniczej Solidarności. - Nowe przepisy, które wkrótce zaczną obowiązywać, takie jak "Krajowy Plan w dziedzinie Energii i Klimatu do 2030 roku", tworzy się w op

In [4]:
# Leave news with unique title
news_df_no_duplicates = news_df.dropDuplicates(["title"])

In [7]:
news_df_no_duplicates.head()

Row(title='3 tys. uwag do nowej konstytucji klimatycznej. Biznes wie, czego chce', date='2024-11-26 13:50:00', keywords=['POLITYKA ENERGETYCZNA', 'PSE', 'TGPE', 'TOKARSKI STANISŁAW', 'ELEKTROWNIE WĘGLOWE', 'ONICHIMOWSKI GRZEGORZ', 'SZULC WALDEMAR', 'TOWARZYSTWO GOSPODARCZE POLSKIE ELEKTROWNIE', 'RYNEK MOCY', 'KIG', 'KPEIK', 'MINISTERSTWO KLIMATU I ŚRODOWISKA', 'MOCE DYSPOZYCYJNE'], is_premium=False, source_site='wnp.pl', url='https://www.wnp.pl/gazownictwo/3-tys-uwag-do-nowej-konstytucji-klimatycznej-biznes-wie-czego-chce,891670.html', text='Do projektu Krajowego Planu w dziedzinie Energii i Klimatu do 2030 r. zgłoszono prawie 3 tys. uwag. Krajowa Izba Gospodarcza podkreśla potrzebę zapewnienia mocy sterowalnych w polskim systemie elektroenergetycznymi. Zdecydowanie wybrzmiewa  postulat kontynuacji rynku mocy. Dyskusja na temat projektu aktualizacji Krajowego Planu w dziedzinie Energii i Klimatu do 2030 r. (KPEiK) była głównym punktem XIII Meetingu Gospodarczego KIG, który odbył się 25

In [5]:
# Leave fows from yfinance with unique timestamp
yfinance_df_no_duplicates = yfinance_df.dropDuplicates(["timestamp"])

In [6]:
# Change date from news from format "YYYY-MM-DD" to "YYY-MM-DDTHH:MM:SS.000000"
from pyspark.sql.functions import col, to_timestamp

news_df_no_duplicates = news_df_no_duplicates.withColumn("date", to_timestamp(col("date"), "yyyy-MM-dd"))

# Rename column "date" to "datetime"
news_df_no_duplicates = news_df_no_duplicates.withColumnRenamed("date", "datetime")

In [7]:
news_df_no_duplicates.head()

Row(title='3 tys. uwag do nowej konstytucji klimatycznej. Biznes wie, czego chce', datetime=datetime.datetime(2024, 11, 26, 0, 0), keywords=['POLITYKA ENERGETYCZNA', 'PSE', 'TGPE', 'TOKARSKI STANISŁAW', 'ELEKTROWNIE WĘGLOWE', 'ONICHIMOWSKI GRZEGORZ', 'SZULC WALDEMAR', 'TOWARZYSTWO GOSPODARCZE POLSKIE ELEKTROWNIE', 'RYNEK MOCY', 'KIG', 'KPEIK', 'MINISTERSTWO KLIMATU I ŚRODOWISKA', 'MOCE DYSPOZYCYJNE'], is_premium=False, source_site='wnp.pl', url='https://www.wnp.pl/gazownictwo/3-tys-uwag-do-nowej-konstytucji-klimatycznej-biznes-wie-czego-chce,891670.html', text='Do projektu Krajowego Planu w dziedzinie Energii i Klimatu do 2030 r. zgłoszono prawie 3 tys. uwag. Krajowa Izba Gospodarcza podkreśla potrzebę zapewnienia mocy sterowalnych w polskim systemie elektroenergetycznymi. Zdecydowanie wybrzmiewa  postulat kontynuacji rynku mocy. Dyskusja na temat projektu aktualizacji Krajowego Planu w dziedzinie Energii i Klimatu do 2030 r. (KPEiK) była głównym punktem XIII Meetingu Gospodarczego KIG